In [3]:
import nfl_func

In [4]:
roster_columns = ['team', 'position', 'depth_chart_position', 'jersey_number','status', 'player_name', 'first_name', 'last_name', 'birth_date', 'height', 'weight', 'player_id', 'years_exp']
roster_data = nfl_func.get_roster_data(2023, roster_columns)
roster_data

,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,player_id,years_exp
0,PHI,OL,T,74.0,CUT,Bernard Williams,Bernard,Williams,NaT,80.0,286,00-0017724,29
1,SEA,OL,T,70.0,INA,Jason Peters,Jason,Peters,1982-01-22,76.0,328,00-0022531,19
2,NYJ,QB,QB,8.0,INA,Aaron Rodgers,Aaron,Rodgers,1983-12-02,74.0,225,00-0023459,18
3,ARI,K,K,5.0,ACT,Matt Prater,Matt,Prater,1984-08-10,70.0,201,00-0023853,17
4,CHI,TE,TE,84.0,ACT,Marcedes Lewis,Marcedes,Lewis,1984-05-19,78.0,267,00-0024243,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3085,CHI,TE,TE,81.0,CUT,Lachlan Pitts,Lachlan,Pitts,NaT,78.0,255,00-0039188,0
3086,NE,OL,T,72.0,CUT,Micah Vanterpool,Micah,Vanterpool,NaT,78.0,315,00-0039189,0
3087,MIN,RB,RB,46.0,CUT,Aaron Dykes,Aaron,Dykes,NaT,71.0,202,00-0039191,0
3088,PHI,DL,DT,67.0,CUT,Caleb Sanders,Caleb,Sanders,NaT,73.0,270,00-0039192,0


In [5]:
all_players = nfl_func.get_players()
player_columns = ['status', 'display_name', 'first_name', 'last_name', 'gsis_id', 'birth_date', 'college_name', 'position_group', 'position', 'height', 'weight', 'years_of_experience', 'team_abbr', 'current_team_id', 'football_name', 'entry_year', 'rookie_year', 'draft_number', 'college_conference']
all_players = all_players[player_columns]

nfl_data_py contains the following functions

cache_pbp

clean_nfl_data

import_combine_data

import_contracts

import_depth_charts

import_draft_picks

import_draft_values

import_ids

import_injuries

import_ngs_data

import_officials

import_pbp_data

import_pfr

import_players

import_qbr

import_rosters

import_sc_lines

import_schedules

import_seasonal_data

import_team_desc

import_weekly_data

import_win_totals

see_pbp_cols

see_weekly_cols
